In [3]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
from flowermd.library import PolyEthylene, OPLS_AA
from flowermd import Pack
from flowermd.modules.welding import SlabSimulation, Interface, WeldSimulation
from cmeutils.visualize import FresnelGSD
import gsd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import unyt as u
molecule = PolyEthylene(num_mols=30, lengths=12)
cpu = hoomd.device.CPU() # So I don't get a CUDA error on local machine


# Create a slab
system = Pack(
    molecules=molecule,
    density=1.1 * u.g/u.cm**3,
)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA(), auto_scale=True,remove_charges=True, remove_hydrogens=True)
sim = SlabSimulation.from_system(
    system=system,
    interface_axis=(1,0,0),
    gsd_file_name="slab_x_interface.gsd", device = cpu
)
sim.forces
for wall in sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", sim.box_lengths_reduced)
target_box = flowermd.utils.get_target_box_mass_density(density=1.2*u.g/u.cm**3, mass=sim.mass.to("g"))

sim.run_update_volume(final_box_lengths=target_box, n_steps=5e4, kT=5.0, period=100, tau_kt=0.001)
sim.run_NVT(kT=5.0, n_steps=4e4, tau_kt=0.001)
sim.remove_walls(wall_axis=(1,0,0))
sim.pickle_forcefield()
sim.flush_writers()
sim_viewer = FresnelGSD(gsd_file="slab_x_interface.gsd", view_axis=(0, 1, 0), frame=-1)
sim_viewer.view()

# Create interface from the slab
interface = Interface(gsd_files=["slab_x_interface.gsd"], interface_axis=(1, 0, 0), gap=0.05)
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])# Create interface from the slab
interface = Interface(gsd_files=["slab_x_interface.gsd"], interface_axis=(1, 0, 0), gap=0.05)
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

# Running a welding simulation
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)

# Let's see what is stored in this pickle file
for force in hoomd_forces:
    print(force)

weld_sim = WeldSimulation(
    initial_state=interface.hoomd_snapshot,
    forcefield=hoomd_forces,
    interface_axis=(1, 0, 0),
    gsd_file_name="weld.gsd",
    log_file_name="weld_log.txt",
    log_write_freq=500,
    dt=0.0003, device = cpu
)
weld_sim.forces
for wall in weld_sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", weld_sim.box_lengths_reduced)
weld_sim.run_NVT(kT=10.0, n_steps=7e4, tau_kt=0.001)
cooling_ramp = weld_sim.temperature_ramp(n_steps=2e4, kT_start=10.0, kT_final=2.0)
weld_sim.run_NVT(kT=cooling_ramp, n_steps=2e4, tau_kt=0.001)
weld_sim.save_restart_gsd("weld_restart.gsd")
weld_sim.flush_writers()


No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Plane(origin=_HOOMDTuple(17.74683380126953, 0.0, 0.0), normal=_HOOMDTuple(-1.0, 0.0, 0.0), 
Plane(origin=_HOOMDTuple(-17.74683380126953, -0.0, -0.0), normal=_HOOMDTuple(1.0, 0.0, 0.0), 

Simulation volume: [35.4936676 35.4936676 35.4936676]
Step 5500 of 50000; TPS: 4423.0; ETA: 0.2 minutes
Step 11000 of 50000; TPS: 4874.64; ETA: 0.1 minutes
Step 16500 of 50000; TPS: 4940.66; ETA: 0.1 minutes
Step 22000 of 50000; TPS: 4936.3; ETA: 0.1 minutes
Step 27500 of 50000; TPS: 4834.8; ETA: 0.1 minutes
Step 33000 of 50000; TPS: 4763.1; ETA: 0.1 minutes
Step 38500 of 50000; TPS: 4597.92; ETA: 0.0 minutes
Step 44000 of 50000; TPS: 4268.27; ETA: 0.0 minutes
Step 49500 of 50000; TPS: 3580.85; ETA: 0.0 minutes
Step 4999 of 40000; TPS: 1024.24; ETA: 0.6 minutes
Step 10499 of 40000; TPS: 1037.24; ETA: 0.5 minutes
Step 15999 of 40000; TPS: 1035.38; ETA: 0.4 minutes
Step 21499 of 40000; TPS: 1035.86; 

In [4]:
# Create interface from the slab
interface = Interface(gsd_files=["slab_x_interface.gsd"], interface_axis=(1, 0, 0), gap=0.05)
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

# Running a welding simulation
# Open and load the forcefield picke file
with open("forcefield.pickle", "rb") as f:
    hoomd_forces = pickle.load(f)

weld_sim = WeldSimulation(
    initial_state=interface.hoomd_snapshot,
    forcefield=hoomd_forces,
    interface_axis=(1, 0, 0),
    gsd_file_name="weld.gsd",
    log_file_name="weld_log.txt",
    log_write_freq=500,
    dt=0.0003, device = cpu
)
weld_sim.forces
for wall in weld_sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", weld_sim.box_lengths_reduced)
weld_sim.run_NVT(kT=10.0, n_steps=7e4, tau_kt=0.001)
cooling_ramp = weld_sim.temperature_ramp(n_steps=2e4, kT_start=10.0, kT_final=2.0)
weld_sim.run_NVT(kT=cooling_ramp, n_steps=2e4, tau_kt=0.001)
weld_sim.save_restart_gsd("weld_restart.gsd")
weld_sim.flush_writers()


<class 'gsd.hoomd.Frame'>
Slab number of particles: 720
Interface number of particles: 1440

Slab box lengths: [6.89579988 6.89579988 6.89579988]
Interface box lengths: [12.8416  6.8958  6.8958]
Initializing simulation state from a gsd.hoomd.Frame.
Plane(origin=_HOOMDTuple(6.42080020904541, 0.0, 0.0), normal=_HOOMDTuple(-1.0, 0.0, 0.0), 
Plane(origin=_HOOMDTuple(-6.42080020904541, -0.0, -0.0), normal=_HOOMDTuple(1.0, 0.0, 0.0), 

Simulation volume: [12.84160042  6.89580011  6.89580011]
Step 5250 of 70000; TPS: 450.53; ETA: 2.4 minutes
Step 10500 of 70000; TPS: 454.21; ETA: 2.2 minutes
Step 15750 of 70000; TPS: 456.02; ETA: 2.0 minutes
Step 21000 of 70000; TPS: 458.35; ETA: 1.8 minutes
Step 26250 of 70000; TPS: 458.77; ETA: 1.6 minutes
Step 31500 of 70000; TPS: 459.8; ETA: 1.4 minutes
Step 36750 of 70000; TPS: 460.05; ETA: 1.2 minutes
Step 42000 of 70000; TPS: 460.16; ETA: 1.0 minutes
Step 47250 of 70000; TPS: 460.76; ETA: 0.8 minutes
Step 52500 of 70000; TPS: 460.76; ETA: 0.6 minutes
S

In [ ]:
sim_viewer = FresnelGSD(gsd_file="weld.gsd", view_axis=(0, 1, 0), frame=0, height=12)
weld_colors = np.zeros_like(sim_viewer.positions)
weld_colors[:weld_colors.shape[0]//2 + 1] = np.array([0.5, 0.25, 0.5])
weld_colors[weld_colors.shape[0]//2 + 1:] = np.array([0.5, 0.1, 0.1])
sim_viewer.colors = weld_colors

sim_viewer.frame = -1
sim_viewer.height = 12
sim_viewer.view_axis = (0, 1, 0)
sim_viewer.view(width=500, height=500)